In [1]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
#import seaborn as sns
import numpy as np
import pandas as pd


SP500=pd.read_csv('SP500_new.csv')
Nasdaq=pd.read_csv('Nasdaq_new.csv')
DJI=pd.read_csv('DJI_new.csv')
DAX=pd.read_csv('DAX_new.csv')
Paris=pd.read_csv('Paris_new.csv')
Tokyo=pd.read_csv('Tokyo_new.csv')
HongKong=pd.read_csv('HongKong_new.csv')
Aus=pd.read_csv('Aus_new.csv')

Data Preprocess

In [2]:
Target=SP500['Adj Close'].pct_change(1).shift(-1)

In [3]:
datasets=[SP500,Nasdaq,DJI,DAX,Paris,Tokyo,HongKong,Aus]
namesets=['SP500','Nasdaq','DJI','DAX','Paris','Tokyo','HongKong','Aus']

In [4]:
def addFeatures(datasets,DaysRet,DaysRetMovAvg,DaysLags):
    """
    compute dataset's DaysReturns,DaysReturnMovAvg and DaysLags
    DaysReturnMovAvg must >=2
    """
    Max=max(DaysRet,DaysRetMovAvg,DaysLags+1)
    for i in range(len(datasets)):
        dataset=datasets[i]
        nameset=namesets[i]
        for i in range(1,DaysRet+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRet'] = dataset['Adj Close'].pct_change(i)
        for i in range(2,DaysRetMovAvg+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRetMovAvg'] = pd.rolling_mean(dataset[nameset+'_'+str(1)+'_'+'DaysRet'], i)
        for i in range(1,DaysLags+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysLags'] = dataset[nameset+'_'+str(1)+'_'+'DaysRet'].shift(i)
        dataset=dataset[Max:]
    return Max

In [5]:
Max=addFeatures(datasets,9,9,9)

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
C:\Program Files\Anacond

In [6]:
def mergedataset(datasets):
    useless=['Date','Open','High','Low','Close','Adj Close','Volume']
    for i in range(len(datasets)):
        datasets[i]=datasets[i].drop(useless, axis = 1)
    new_dataset = pd.concat(datasets,axis=1)
    return new_dataset

In [7]:
features=mergedataset(datasets)

In [8]:
features=features[Max:-1]
Target=Target[Max:-1]

transfer Target data to binary

In [9]:
Target[Target > 0] = 1
Target[Target <= 0] = 0

Shuffle and Split Data

In [10]:
X_train=features[:-500]
X_test=features[-500:]
y_train=Target[:-500]
y_test=Target[-500:]


#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(features,Target, test_size = 0.2, random_state = 0)

# Modeling

In [11]:
from sklearn.metrics import accuracy_score
def performTimeSeriesCV(X_train, y_train, number_folds, algorithm,params_fix, params):
    """
    This is cumtomised gridsearch function for time series problem.the tradional grid search randomly spit the dataset which is not suitable for time series problem as it stricctly requires using old data to predict new data! Thus,I make this time series gridsearch function,it goes in this way:
    1.Split train set in " number_folds" consecutive time folds.
    2.Then, in order not lo lose the time information, perform the following steps:
    3.Train on fold 1 –>  Test on fold 2
    4.Train on fold 1+2 –>  Test on fold 3
    5.Train on fold 1+2+3 –>  Test on fold 4
    .
    .
    .
    n.Train on fold 1+2+3+...+ "number_folds-1" –>  Test on fold "number_folds" 
    n+1.Compute the average of the accuracies of the "number_folds-1" test folds 
    """
    
    a=params_fix
    total={}
    for i in range(len(list(params.values())[0])):
        for j in range(len(list(params.values())[1])):
            a[list(params.keys())[0]]=list(params.values())[0][i]
            a[list(params.keys())[1]]=list(params.values())[1][j]
            k = int(np.floor(float(X_train.shape[0]) / number_folds))
            accuracies = np.zeros(number_folds-1)
            for t in range(2, number_folds + 1):
                split = float(t-1)/t
         
                X = X_train[:(k*t)]
                y = y_train[:(k*t)]

                index = int(np.floor(X.shape[0] * split))
           
                X_trainFolds = X[:index]        
                y_trainFolds = y[:index]

                X_testFold = X[(index + 1):]
                y_testFold = y[(index + 1):]
                
                reg=algorithm(**a)
                reg.fit(X_trainFolds, y_trainFolds)
                
                y_predict=reg.predict(X_testFold)
                accuracies[t-2]=accuracy_score(y_testFold,y_predict)
            #print(accuracies.mean())
            info=[]
            info.append(list(params.values())[0][i])
            info.append(list(params.values())[1][j])
            total[str(accuracies.mean())]=info
    compare=[]
    for i in total.keys():
        compare.append(float(i))
    Best_accu=max(compare)
    print('Best accuracy:',Best_accu)
    print(list(params.keys())[0],':',total[str(Best_accu)][0])
    print(list(params.keys())[1],':',total[str(Best_accu)][1])
    fin_params=params_fix
    fin_params[list(params.keys())[0]]=total[str(Best_accu)][0]
    fin_params[list(params.keys())[1]]=total[str(Best_accu)][1]
    return fin_params

1.RandomForestClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
para=performTimeSeriesCV(X_train, y_train, 10, RandomForestClassifier,{},{'n_estimators':[120,150],'min_samples_split':[2]})

TypeError: 'dict_values' object does not support indexing

In [43]:
print(para)

{'n_estimators': 150, 'min_samples_split': 2}


In [49]:
model=RandomForestClassifier(n_estimators=120)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=120, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [50]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.516


2 KNeighborsClassifier

In [13]:
from sklearn.neighbors import KNeighborsClassifier
para=performTimeSeriesCV(X_train, y_train, 10, KNeighborsClassifier,{'n_neighbors':[23,24,25,26,27,28],'leaf_size':[30]})

TypeError: performTimeSeriesCV() missing 1 required positional argument: 'params'

In [217]:
model=KNeighborsClassifier(n_neighbors=25)
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
           weights='uniform')

In [218]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.532


3 SVC

In [24]:
from sklearn.svm import SVC
performTimeSeriesCV(X_train, y_train, 10, SVC,{'C':[200,300,400,500,600,700,800,900],'degree':[200,300,400,500,600,700,800,900]})


('Best accuracy:', 0.542251786154)
('C', ':', 400)
('degree', ':', 900)


In [27]:
from sklearn.svm import SVC
model=SVC(C=400,degree=900)
model.fit(X_train, y_train)

SVC(C=400, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=900, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.512


4 AdaBoostClassifier

In [29]:
from sklearn.ensemble import  AdaBoostClassifier
performTimeSeriesCV(X_train, y_train, 10, AdaBoostClassifier,{'n_estimators':[72,73,74,75,76,77,78],'learning_rate':[2,2.5,3,3.5,4,4.5,5]})

('Best accuracy:', 0.535353535354)
('n_estimators', ':', 77)
('learning_rate', ':', 4)


In [34]:
model=AdaBoostClassifier(n_estimators=77,learning_rate=4)
model.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=4,
          n_estimators=77, random_state=None)

In [35]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.478


5 GradientBoostingClassifier

5.1 benchmark

In [24]:
model=GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [25]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.508


5.2 Fix learning rate and number of estimators for tuning tree-based parameters

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier(learning_rate=0.1, min_samples_split=100,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10)
performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'min_samples_split':100,'min_samples_leaf':50,'max_depth':8,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'n_estimators':[20,30,40,50,60,70,80],'learning_rate':[0.1]})

('Best accuracy:', 0.522296132052)
('n_estimators', ':', 30)
('learning_rate', ':', 0.1)


In [37]:
params={'n_estimators':30,'learning_rate':0.1,'min_samples_split':100,'min_samples_leaf':50,'max_depth':8,'max_features':'sqrt','subsample':0.8,'random_state':10}
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=100, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [38]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.512


5.3 Tune max_depth and num_samples_split

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[5,7,9,11,13,15],'min_samples_split':[200,400,600,800,1000]})

TypeError: 'dict_values' object does not support indexing

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[6,7,8],'min_samples_split':[50,100,150,200]})

('Best accuracy:', 0.525006159153)
('min_samples_split', ':', 150)
('max_depth', ':', 8)


In [66]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'max_depth':[8],'min_samples_split':[100,120,130,140,150]})

('Best accuracy:', 0.532150776053)
('min_samples_split', ':', 120)
('max_depth', ':', 8)


In [67]:
print(params)

{'learning_rate': 0.1, 'min_samples_leaf': 50, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120, 'max_depth': 8}


In [68]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [69]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.528


5.4 Tune min_samples_leaf

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'n_estimators':30,'learning_rate':0.1,'min_samples_leaf':50,'max_features':'sqrt','subsample':0.8,'random_state':10},
                    {'min_samples_split':[120],'min_samples_leaf':[30,40,50,60,70]})

('Best accuracy:', 0.531165311653)
('min_samples_split', ':', 120)
('min_samples_leaf', ':', 40)


In [17]:
print(params)

{'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120}


In [18]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=40,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [19]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.536


5.5 Tune max_features

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120},
                    {'min_samples_leaf':[40],'max_features':[7,9,11,13,15,17,19,21]})

('Best accuracy:', 0.526730721853)
('max_features', ':', 19)
('min_samples_leaf', ':', 40)


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120},
                    {'min_samples_leaf':[40],'max_features':[17,18,19,20,21,22,23]})

('Best accuracy:', 0.526730721853)
('max_features', ':', 19)
('min_samples_leaf', ':', 40)


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120},
                    {'min_samples_leaf':[40],'max_features':[7,19,27,37,47,57,67,77,87]})

('Best accuracy:', 0.527469820153)
('max_features', ':', 57)
('min_samples_leaf', ':', 40)


In [41]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 'sqrt', 'min_samples_split': 120},
                    {'min_samples_leaf':[40],'max_features':[7,19,27,37,47,57,67,77,87,97,107,117,127]})

('Best accuracy:', 0.527469820153)
('max_features', ':', 57)
('min_samples_leaf', ':', 40)


In [42]:
print(params)

{'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120}


In [43]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=57, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=40,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [44]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.538


5.6 Tuning subsample

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120},
                    {'subsample':[0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89],'max_features':[57]})




('Best accuracy:', 0.534368070953)
('max_features', ':', 57)
('subsample', ':', 0.88)


In [53]:
from sklearn.ensemble import GradientBoostingClassifier
params=performTimeSeriesCV(X_train, y_train, 10,
                    GradientBoostingClassifier,
                    {'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120},
                    {'subsample':[0.77,0.78,0.79,0.80,0.81,0.82,0.83],'max_features':[57]})




('Best accuracy:', 0.527469820153)
('max_features', ':', 57)
('subsample', ':', 0.8)


In [54]:
print(params)

{'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120}


In [55]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=57, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=40,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [56]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.538


In [ ]:
5.7 lower learning rate

In [99]:
params={'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120}



In [100]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=57, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=40,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [101]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.538


6 QuadraticDiscriminantAnalysis

In [18]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model=QuadraticDiscriminantAnalysis()
model.fit(X_train, y_train)

/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariances=False, tol=0.0001)

In [19]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.496


In [63]:
model=GradientBoostingClassifier(**params)
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=120, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [ ]:
7 SGDClassifier

In [20]:
from sklearn.linear_model import SGDClassifier
performTimeSeriesCV(X_train, y_train, 10, SGDClassifier,{'n_iter':[3,5,7,10,13,15],'l1_ratio':[0.13,0.14,0.15,0.16,0.17,0.18]})

('Best accuracy:', 0.539049026854)
('n_iter', ':', 10)
('l1_ratio', ':', 0.17)


In [21]:
model=SGDClassifier(n_iter=10,l1_ratio=0.17)
model.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.17,
       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [22]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.492


In [ ]:
RandomForestClassifier(n_estimators=120)           0.53
KNeighborsClassifier(n_neighbors=25)               0.532
SVC(C=400,degree=900)                              0.532
AdaBoostClassifier(n_estimators=77,learning_rate=4)0.478

# Stacking models

In [103]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score


In [104]:
'''Train the stacked models then predict the test data'''
#ORIGINAL
class StackingEstimator(BaseEstimator, TransformerMixin):
        
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

In [109]:
from sklearn.ensemble import RandomForestClassifier
params={'learning_rate': 0.1, 'min_samples_leaf': 40, 'n_estimators': 30, 'subsample': 0.8, 'random_state': 10, 'max_features': 57, 'min_samples_split': 120}
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(n_estimators=120)),
    StackingEstimator(estimator=GradientBoostingClassifier(**params)),
    RandomForestClassifier(n_estimators=120)

)

In [110]:
stacked_pipeline.fit(X_train, y_train)
y_predict=stacked_pipeline.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.528
